# Draw a Molecule

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.data.structure import StructureData

import numpy as np
from numpy.linalg import norm
import scipy.stats

import ipywidgets as ipw
from IPython.display import display, clear_output
import nglview
import StringIO
from tempfile import NamedTemporaryFile

import ase.io
from molview_ipywidget import MolViewWidget

## Step 1: Draw Molecule

In [ ]:
molview = MolViewWidget()

def on_mol_change(c):
    global atoms
    with mol_out:
        clear_output()
        tmp = NamedTemporaryFile(suffix=".mol")
        f = open(tmp.name, "w")
        f.write(molview.value)
        f.close()
        atoms = ase.io.read(tmp.name)
        tmp.close()
        fix_scale(atoms)
        atoms.center(vacuum=inp_vac.value)
        atoms.pbc = (True,True,True)
        update_view()

mol_out = ipw.Output()
molview.observe(on_mol_change, names='value')
display(molview, mol_out)

In [ ]:
def fix_scale(atoms):
    # set bounding box as cell
    cx = 1.5 * (np.amax(atoms.positions[:,0]) - np.amin(atoms.positions[:,0]))
    cy = 1.5 * (np.amax(atoms.positions[:,1]) - np.amin(atoms.positions[:,1]))
    cz = 1.5 * (np.amax(atoms.positions[:,2]) - np.amin(atoms.positions[:,2]))
    atoms.cell = (cx, cy, cz)
    atoms.pbc = (True,True,True)
    
    # calculate all distances between C atoms
    c_atoms = [i for i, s in enumerate(atoms.get_chemical_symbols()) if s=="C"]
    n = len(c_atoms)
    if n<2:
        raise Exception("Can not determine scale without C-C bonds.")
    dists = np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            dists[i,j] = norm(atoms[c_atoms[i]].position - atoms[c_atoms[j]].position)
            
    # find bond distances to closest neighbor
    dists += np.diag([np.inf]*n) # don't consider diagonal
    bonds = np.amin(dists, axis=1)
    print(bonds)
    # average bond distance
    avg_bond = float(scipy.stats.mode(bonds)[0])
    
    # scale box to match equilibrium carbon-carbon bond distance
    cc_eq = 1.4313333333
    s = cc_eq / avg_bond
    print("Scaled positions by: %f"%s)
    atoms.set_cell([s*cx, s*cy, cz], scale_atoms=True)
    atoms.center()

In [ ]:
def update_view():
    global atoms
    viewer.remove_unitcell()
    if hasattr(viewer, "component_0"):
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center_view()
       
    tmp = StringIO.StringIO()
    atoms.write(tmp, format="xyz")
    coords.value = "<pre>"+tmp.getvalue()+"</pre>"
    tmp.close()
    
viewer = nglview.NGLWidget()
coords = ipw.HTML()
display(ipw.VBox([viewer, coords]))

## Step 2: Define Cell

In [ ]:
def on_vac_change(c):
    global atoms
    atoms.center(vacuum=inp_vac.value)
    atoms.pbc = (True,True,True)
    update_view()

inp_vac = ipw.FloatSlider(value=2.5, min=0.1, max=10.0, continuous_update=False)
lab_vac = ipw.Label(u'Vacuum [Å]:')
inp_vac.observe(on_vac_change, "value")
display(ipw.HBox([lab_vac, inp_vac]))

## Step 3: Store in AiiDA Database

In [ ]:
def on_click_store(b):
    global atoms
    with store_out:
        clear_output()
        s = StructureData(ase=atoms)
        s.description = inp_descr.value
        s.store()
        print("Stored in AiiDA: "+repr(s))

store_out = ipw.Output()
inp_descr = ipw.Text(placeholder="Description (optional)")
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
display(ipw.HBox([btn_store, inp_descr]), store_out)